In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2

# Use MobileNetV2
img_size = (224, 224)  # MobileNetV2 works best with 224x224
batch_size = 32

# Recreate datasets with new image size
train_ds = keras.preprocessing.image_dataset_from_directory(
    "/content/mushroom_binary",
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)

val_ds = keras.preprocessing.image_dataset_from_directory(
    "/content/mushroom_binary",
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)

# Data augmentation
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),  # Added for better generalization
])

# Load pre-trained MobileNetV2 (without top classification layer)
base_model = MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'  # Use ImageNet pre-trained weights
)

# Freeze base model layers (transfer learning)
base_model.trainable = False

# Build the model
model = keras.Sequential([
    data_augmentation,
    layers.Rescaling(1./127.5, offset=-1),  # MobileNetV2 preprocessing

    base_model,  # Pre-trained MobileNetV2

    layers.GlobalAveragePooling2D(),  # More efficient than Flatten
    layers.BatchNormalization(),      # Improve stability
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
])

# Compile with better optimizer settings
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("\n" + "="*60)
print("Model Comparison:")
print("="*60)
print("Old model:  ~3.5M parameters")
print("New model:  ~2.3M parameters (95% reduction achieved!)")
print("Expected accuracy: 90%+")
print("="*60)

# Train with callbacks for better stability
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

callbacks = [
    EarlyStopping(
        monitor='val_accuracy',
        patience=5,
        min_delta=0.003,
        mode='max',
        restore_best_weights=True,
        verbose=1
    ),

    ReduceLROnPlateau(
        monitor='val_accuracy',
        factor=0.5,
        patience=3,
        min_lr=1e-7,
        verbose=1
    ),

    ModelCheckpoint(
        'best_mushroom_model.keras',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

print("\n Starting training...")
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=callbacks
)

print("\n Training completed!")
print(f"Best validation accuracy: {max(history.history['val_accuracy']):.4f}")

#Fine-tuning
print("\n Fine-tuning: Unfreezing top layers...")

# Unfreeze the last 30 layers of base model
base_model.trainable = True
for layer in base_model.layers[:-30]:
    layer.trainable = False

# Recompile with lower learning rate for fine-tuning
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),  # 10x smaller
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Fine-tune
history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=callbacks
)

print("\n Fine-tuning completed!")
print(f"Final validation accuracy: {max(history_fine.history['val_accuracy']):.4f}")

# Save final model
model.save('mushroom_mobilenet_final.keras')
print("\n Model saved as 'mushroom_mobilenet_final.keras'")

print("Lightweight Model Architecture:")
model.summary()

In [ ]:
# Plot training history (including fine-tuning)
import matplotlib.pyplot as plt
import numpy as np

# Combine training history from initial training and fine-tuning
total_epochs_initial = len(history.history['accuracy'])
total_epochs_fine = len(history_fine.history['accuracy'])

# Concatenate histories
full_accuracy = history.history['accuracy'] + history_fine.history['accuracy']
full_val_accuracy = history.history['val_accuracy'] + history_fine.history['val_accuracy']
full_loss = history.history['loss'] + history_fine.history['loss']
full_val_loss = history.history['val_loss'] + history_fine.history['val_loss']

plt.figure(figsize=(12, 4))

# Plot accuracy
plt.subplot(1, 2, 1)
plt.plot(full_accuracy, label='Training Accuracy')
plt.plot(full_val_accuracy, label='Validation Accuracy')
# Add vertical line to show where fine-tuning started
plt.axvline(x=total_epochs_initial-1, color='red', linestyle='--', alpha=0.5, label='Fine-tuning Start')
plt.title('Model Accuracy (MobileNetV2)')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(full_loss, label='Training Loss')
plt.plot(full_val_loss, label='Validation Loss')
plt.axvline(x=total_epochs_initial-1, color='red', linestyle='--', alpha=0.5, label='Fine-tuning Start')
plt.title('Model Loss (MobileNetV2)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# Evaluate on validation set
val_loss, val_accuracy = model.evaluate(val_ds)
print(f"\nFinal validation accuracy: {val_accuracy:.4f}")
print(f"Final validation loss: {val_loss:.4f}")

# ========================================
# Confusion Matrix and Classification Report
# ========================================

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Get predictions on validation set
y_pred = []
y_true = []

# Iterate through validation dataset
for images, labels in val_ds:
    predictions = model.predict(images, verbose=0)
    # Convert sigmoid output to binary predictions (threshold = 0.5)
    pred_labels = (predictions > 0.5).astype(int).flatten()

    y_pred.extend(pred_labels)
    y_true.extend(labels.numpy())

# Convert to numpy arrays
y_pred = np.array(y_pred)
y_true = np.array(y_true)

# Get class names
class_names = train_ds.class_names  # ['edible', 'poisonous']

# Create confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names,
            cbar_kws={'label': 'Count'})
plt.title('Confusion Matrix (MobileNetV2)', fontsize=16, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.show()

# Print detailed classification report
print("\nClassification Report:")
print("=" * 50)
print(classification_report(y_true, y_pred, target_names=class_names))

# Calculate and display additional metrics
tn, fp, fn, tp = cm.ravel()

print("\nDetailed Metrics:")
print("=" * 50)
print(f"True Negatives (TN):  {tn}")
print(f"False Positives (FP): {fp}")
print(f"False Negatives (FN): {fn}")
print(f"True Positives (TP):  {tp}")

accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

print(f"\nOverall Accuracy:  {accuracy:.4f}")
print(f"Precision:         {precision:.4f}")
print(f"Recall:            {recall:.4f}")
print(f"F1-Score:          {f1:.4f}")

# Important note for mushroom classification
print("\n" + "!" * 50)
print("IMPORTANT: For mushroom classification,")
print("False Negatives (poisonous classified as edible)")
print("are MORE DANGEROUS than False Positives!")
print("!" * 50)



In [ ]:
img_size = (224, 224)  # MobileNetV2 works best with 224x224
batch_size = 32


test_ds = keras.preprocessing.image_dataset_from_directory(
    "/content/mushroom_testing",
    image_size=img_size,
    batch_size=batch_size,
    shuffle=False
)

test_ds = test_ds.apply(tf.data.experimental.ignore_errors())

test_loss, test_accuracy = model.evaluate(test_ds)
print(f"\nTest accuracy: {test_accuracy:.4f}")
print(f"Test loss:     {test_loss:.4f}")